### Check dependencies

In [1]:
import importlib

libraries = [
    "tensorflow",
    "numpy",
    "spektral",
    "matplotlib",
    "pandas",
    "sklearn"
]

for lib in libraries:
    try:
        if not importlib.util.find_spec(lib):
            raise ImportError("Package '%s' is not installed" % lib)
    except ImportError as e:
        print(e)

In [ ]:
!pip install tensorflow

### Soil moisture data preprocessing

The SMAP data has already been cropped into numpy arrays for the three regions of interest:
- Italy (`Italy_data.npy`)
- California (`California_data.npy`)
- South Africa (`SouthAfrica_data.npy`)

and labels:

- Italy (`labels/Italy.npy`)
- California (`labels/California.npy`)
- South Africa (`labels/SouthAfrica.npy`)

Using `load_italy()` as example, write functions to do the same for California/South Africa

In [ ]:
import numpy as np
import pandas as pd
import os
import completed

def load_italy():
    return np.load("Italy_data.npy"), np.load(os.path.join("labels", "Italy.npy"))

def load_calif():
    """
    TASK 1: Write function to read and return California data (can just copy load_italy() and change file name)
    """
    raise NotImplementedError
    return

def load_southafrica():
    """
    TASK 2: Write function to read and return SouthAfrica data (can just copy load_italy() and change file name)
    """
    raise NotImplementedError
    return

def load_data(area):
    regions = ["italy", "california", "southafrica"]
    if area not in regions:
        raise ValueError("'region' must be one of {} but was {}".format(regions, region))
    else:
        if area == "italy":
            return load_italy()
        if area == "california":
            return load_calif()
        if area == "southafrica":
            return load_southafrica()

Let's visualise this data, the following produces a plot of the data at 10 time stamps (each 5 days after the previous):

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

region = "italy"
data, label_map = load_data(region)

for i in range(10):
    fig, ax = plt.subplots()
    ax.matshow(data[..., i * 10])
    plt.show()

In [ ]:
print("Max: %s" % np.nanmax(data))
print("Min: %s" % np.nanmin(data))
print("Missing value ratio: %s" % (np.sum(np.isnan(data)) / np.prod(data.shape)))
print("Data shape: {}".format(data.shape))

**Linearly interpolates to fill missing data**

To fill the missing values, we will use a simple linear interpolation approach.
There is quick method for doing so which uses the `pandas` library to make a `Dataframe` that has linear interpolation functionality.

This function (and most of the code) assumes we are working with a 2-dimensional array which has nodes along the first dimension and time along the second (and last) dimension. To do this, we will flatten the spatial dimensions of the SMAP data.

In [ ]:
def lin_interp(data_values):
    """
    data: array (Last axis must be time)
    """
    ndim = data_values.ndim
    in_shape = data_values.shape
    if ndim > 2:
        data_values = data_values.reshape(-1, data_values.shape[-1])
    return pd.DataFrame(data_values.T).interpolate(method="linear", limit_direction="forward").interpolate(method="linear",
                                                                                                    limit_direction="backward").values.T

To show this in operation, here is an example, before and after interpolation. Note that if there are no observed values over time the `nan` values aren't filled.

In [ ]:
test_array = np.array([[1., np.nan, np.nan, 7.], [np.nan, 2., 3., np.nan], [np.nan, np.nan, np.nan, np.nan], [np.nan, 3., np.nan, 5]])

print("Before:")
print(test_array)
print()
print("After:")
print(lin_interp(test_array))

**Write a function to normalize the data between 0 and 1 (if not already)**

Generally speaking, it is common practice to normalize features in the data before using deep learning methods. This ensures that gradients will be comparable across features and applications which means that there is less need to tune optimization parameters such as learning rates/momentum etc.

We will again assume that the function input will be a 2-dimensional array with #nodes x #timesteps.

In [ ]:
def normalize(data_values):
    """
    TASK 3: Write a function to normalize data to between 0 and 1
    data_values: 2-d numpy array with nodes along first axis and time along second axis
    """
    raise NotImplementedError
    return

In [ ]:
test_array = np.array(np.array([[1,2,3],[4,1,1], [3., 1., 3.5]]))

print("Before:")
print(test_array)
print()
print("After:")
print(normalize(test_array))

In [ ]:
def flatten(data_values):
    """
    TASK 4: Write a function to take the 3-dimensional data (space x space x time) 
    and flatten into a 2-d array with time as second axis
    
    data_values: numpy array (ndim=3)
    
    return:
    data_values: numpy array (ndim=2)
    """
    raise NotImplementedError
    return

**Use our pre-processing functions to prepare our data**

In [ ]:
filled = lin_interp(flatten(data))
normalized = normalize(filled)

Let's revisualize data now it has been interpolated/normalized

In [ ]:
reshaped = normalized.reshape(data.shape)

for i in range(10):
    fig, ax = plt.subplots()
    cm = ax.matshow(normalized.reshape(data.shape)[..., i * 10])
    fig.colorbar(cm, ax=ax)
    plt.show()

Alternatively, we can take specific pixels and look at their time series across the course of the year after interpolation:

In [ ]:
for i in range(10):
    plt.plot(normalized[i * 10])
    plt.show()

For the pixels which still have no data (e.g. pixels representing sea) we will throw them away when we reshape our data into a two-dimensional array. We will want to save this mask though in order to recover the spatial view when visualizing our final results.

In [ ]:
not_seen = np.all(np.isnan(normalized), axis=1)
model_inputs = normalized[~not_seen]  # this gives array of shape (#nodes, #timesteps) without the unobserved regions

plt.matshow(not_seen.reshape(data.shape[:2]))
plt.show()

# Uncertainty-aware graph-based classification

The labels to be used as ground truth for this demonstration can be plotted

In [ ]:
fig, ax = plt.subplots()
cm = ax.matshow(label_map, cmap=plt.cm.Accent)
fig.colorbar(cm, ax=ax)
plt.show()

To flatten our labels and remove the pixels which are not observed, we can use our `not_seen` mask.

This gives `labels` in shape (#nodes,)

In [ ]:
print("Labels shape: {}".format(label_map.shape))

labels = label_map.flatten()[~not_seen]
print("Labels (masked) shape: {}".format(labels.shape))

One-hot encode labels

In [ ]:
from sklearn.preprocessing import OneHotEncoder

labels_oh = OneHotEncoder().fit_transform(labels.reshape(-1, 1)).toarray()

In [ ]:
print("labels_oh.shape: {}".format(labels_oh.shape))
print("first 5 labels:{}".format(labels[:5]))
print("first 5 rows:\n{}".format(labels_oh[:5]))

The inputs and labels should now be of appropriate shapes

In [ ]:
print(model_inputs.shape)
print(labels_oh.shape)

#### Task 1: Write function to perform dataset split

Write a function which will randomly assign each node into one of:

- training
- validation
- testing

based on a predefined ratio.

We will also pass a random seed in order to make the split reproduceable.

In [ ]:
def make_splits(numb_nodes, seed, train_ratio, val_ratio):
    """
    TASK 5: Write a function which will return a numpy array of shape (numb_nodes,)
    The array should contain strings (either "train", "val" or "test") such that
    the proportion of total entries which are "train" is given by `train_ratio`,
    the proportion of total entries which are "val" is given by `val_ratio`,
    and the rest are "test"
    
    numb_nodes: int (number of nodes in graph)
    seed: int (use this to set numpy random seed to make random splits reproduceable)
    train_ratio: float (between 0 and 1)
    val_ration: float (between 0 and 1)
    """
    
    raise NotImplementedError
    return

In [ ]:
print(make_splits(20, 0, .7, .15))

### __Useful functions__

**Compute vacuity uncertainty**

Write a function which computes vacuity uncertainty from Dirichlet parameters.

Assume input is of shape (N, K), i.e. #nodes x #classes.

Output should be of shape (N,)

In [ ]:
def get_vacuity(alphas):
    """
    TASK 6: Write function to compute vacuity uncertainty for each node
    
    alphas: numpy array (2-dimensional) of shape (#nodes, #classes)
    """
    raise NotImplementedError
    return

Simple check to see if our vacuity calculation works

In [ ]:
get_vacuity(np.array([[1,2,3],[1,1,1]]))

**Get belief mass vector from alpha vectors**

In [ ]:
def get_belief(alphas):
    return (alphas - 1) / np.sum(alphas, axis=1, keepdims=True)

Simple check to see if our belief mass calculation works

In [ ]:
get_belief(np.array([[1,2,3],[1,1,1]]))

**Compute dissonance uncertainty**

A function which computes dissonance uncertainty from Dirichlet parameters.

Assume input is of shape (N, K), i.e. #nodes x #classes.

Output should be of shape (N,)

In [ ]:
def dissonance_uncertainty(alphas):
    alphas = np.array(alphas)  # will be used later when output might be a tf.Tensor
    belief = get_belief(alphas)
    dis_un = np.zeros(alphas.shape[0])

    for i in range(alphas.shape[0]):  # for each node
        b = belief[i]  # belief vector
        numerator, denominator = np.abs(b[:, None] - b[None, :]), b[None, :] + b[:, None]
        bal = 1 - np.true_divide(numerator, denominator, where=denominator != 0,
                                 out=np.zeros_like(denominator)) - np.eye(len(b))
        coefficients = b[:, None] * b[None, :] - np.diag(b ** 2)
        denominator = np.sum(
            b[None, :] * np.ones(belief.shape[1]) - np.diag(b), axis=-1, keepdims=True)
        dis_un[i] = (coefficients * np.true_divide(bal, denominator, where=denominator != 0,
                                                   out=np.zeros_like(bal))).sum()

    return dis_un

Simple check to see if our belief mass calculation works

In [ ]:
dissonance_uncertainty(np.array([[20,20,20],[3,2,10]]))

**Get expected value of Dirichlet distribution from alpha vectors**

In [ ]:
def expected_probability(alphas):
    """
    TASK 7: Write function to compute expected probability for each node
    
    alphas: numpy array (2-dimensional) of shape (#nodes, #classes)
    
    return a numpy array (2-dimensional) of shape (#nodes, #classes)
    """
    raise NotImplementedError
    return

Simple check to see if our vacuity calculation works

In [ ]:
expected_probability(np.array([[20, 20, 20],[1, 1, 1], [3, 2, 10]]))

**Function to compute our graph: the connections between pixels/nodes found using k nearest neighbors**

In [ ]:
def get_adj(x, k):
    a = kneighbors_graph(x, k, include_self=False)
    a = a + a.T  # to make graph symmetric (using k neighbours in "either" rather than "mutual" mode)
    a[a > 1] = 1  # get rid of any edges we just made double
    return a

### Prepare data for learning

We will use a package called `spektral` (information found [here](https://graphneural.network/)) which will do much of the heavy lifting for the graph-based deep learning.

To use this library, we will first prepare our data into a `spektral` `Dataset` object in order to be able to make use of the library's functionality.

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import OneHotEncoder
from spektral.data import Dataset, Graph
from spektral.datasets.utils import DATASET_FOLDER


class CustomDataset(Dataset):
    never_seen = None

    def __init__(self,
                 soil_moisture_data,
                 labels,
                 numb_neighbors,
                 seed,
                 region,
                 splits,
                 **kwargs):
        self.x = soil_moisture_data  # inputs
        self.y = labels  # ground truth (one-hot encoded)
        self.numb_neighbs = numb_neighbors  # number of neighbors (for kNN) used to compute graph
        self.mask_tr, self.mask_va, self.mask_te = None, None, None  # dataset splits
        self.numb_op_classes = labels.shape[1]  # number of output classes (labels is one-hot encoded)
        self.seed = seed  # random seed (can be used to compute average performance across different splits)
        self.splits = splits  # train/validation/test splits
        self.region = region  # used for folder naming purposes only
        super().__init__(**kwargs)

    @property
    def path(self):
        # defines a folder in which dataset is saved
        train_ratio = np.sum(self.splits == "train") / np.prod(self.splits.shape)
        val_ratio = np.sum(self.splits == "val") / np.prod(self.splits.shape)
        return os.path.join("spektral",
                            self.__class__.__name__
                            ) + "{}_classes{}_seed{}_train{:.3g}_val{:.3g}_neighbs_{}".format(self.region,
                                                                                      self.numb_op_classes,
                                                                                      self.seed,
                                                                                      train_ratio,
                                                                                      val_ratio,
                                                                                      self.numb_neighbs)

    def download(self):
        x = self.x
        y = self.y
        splits = self.splits

        # get graph adjacency
        a = get_adj(x, k=self.numb_neighbs)  # k is the number of nearest neighbours to look for

        # Create the directory
        if not os.path.isdir("spektral"):
            os.mkdir("spektral")
        os.mkdir(self.path)

        filename = os.path.join(self.path, 'graph')
        
        np.savez(filename, x=x, a=a, y=y)

    def read(self):
        data = np.load(os.path.join(self.path, f'graph.npz'), allow_pickle=True)

        x = data['x'].astype(np.float32)
        a = data['a'].tolist()
        y = data['y'].astype(np.uint8)

        self.mask_tr = (self.splits == "train")
        self.mask_va = (self.splits == "val")
        self.mask_te = (self.splits == "test")

        return [Graph(x=x, a=a, y=y)]

In [ ]:
from spektral.transforms import LayerPreprocess, AdjToSpTensor
from spektral.layers import GCNConv
from spektral.models import GCN

random_seed = 0
sm = CustomDataset(soil_moisture_data = model_inputs,
                   labels = labels_oh,
                   numb_neighbors = 5,
                   seed = random_seed,
                   region = region,
                   splits = make_splits(labels_oh.shape[0], random_seed, train_ratio=.1, val_ratio=.05),
                   transforms=[LayerPreprocess(GCNConv)]
                  )

### Perform learning

Create our Graph Convolutional Network

In [ ]:
import tensorflow as tf

tf.random.set_seed(random_seed)
model = GCN(n_labels=labels_oh.shape[1], channels=32, output_activation=lambda z: tf.exp(z) + 1, l2_reg=0.001)

Our loss function (used to compile network before training)

In [ ]:
class SquareErrorDirichlet(tf.keras.losses.Loss):
    def __init__(self, name="square_error_dirichlet"):
        super().__init__(name=name)

    def call(self, y_true, alpha):
        y_true = tf.cast(y_true, tf.float32)
        strength = tf.reduce_sum(alpha, axis=1, keepdims=True)
        prob = tf.divide(alpha, strength)
        loss = tf.square(prob - y_true) + prob * (1 - prob) / (strength + 1.0)
        return tf.reduce_sum(loss, axis=-1)

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.01),
    loss=SquareErrorDirichlet(),
    weighted_metrics=["acc"]
)

`weight_by_class` function assigns a weight to each node (e.g. masks out test nodes so that they do not contribute to the loss when training)

In [ ]:
def weight_by_class(y, weights):
    samples = y[weights != 0].sum(axis=0)  # count how many
    samples = np.true_divide(len(weights), samples * len(samples[samples != 0]), out=0. * samples, where=samples != 0)
    return (y * samples).max(axis=-1) * weights

In [ ]:
from spektral.data import SingleLoader

weights_tr, weights_va = [weight_by_class(sm[0].y, mask) for mask in [sm.mask_tr, sm.mask_va]]
loader = SingleLoader(sm, sample_weights=weights_tr)
steps_per_epoch = loader.steps_per_epoch
loader_tr = loader.load()
loader_va = SingleLoader(sm, sample_weights=weights_va).load()


history = model.fit(
    loader_tr,
    steps_per_epoch=steps_per_epoch,
    validation_data=loader_va,
    validation_steps=steps_per_epoch,
    epochs=2000,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=50,
                                                restore_best_weights=True),
               tf.keras.callbacks.ModelCheckpoint(os.path.join("GCN.h5"),
                                                  monitor="val_loss", save_best_only=True,
                                                  save_weights_only=True)]
)


In [ ]:
loader_all = SingleLoader(sm, epochs=1)
inputs, outputs = loader_all.__next__()
alpha = np.array(model(inputs, training=True))

`imageify` function used to restore the spatial dimension lost when flattening to remove unobserved regions.

In [ ]:
def imageify(values, shape, never_seen):
    if values.ndim == 1:
        image = np.zeros(shape) * np.nan
    else:
        image = np.zeros(shape + values.shape[1:]) * np.nan
    image[~never_seen.reshape(shape)] = values
    return image

Visualise our results

In [ ]:
predicted_labels_tr = imageify(alpha.argmax(axis=1), data.shape[:2], not_seen)
predicted_labels_tr[imageify(sm.mask_tr, data.shape[:2], not_seen) != True] = np.nan
plt.matshow(predicted_labels_tr, cmap=plt.cm.Accent, vmin=0, vmax=alpha.shape[1] - 1)
plt.title("training nodes")
plt.show()

predicted_labels_va = imageify(alpha.argmax(axis=1), data.shape[:2], not_seen)
predicted_labels_va[imageify(sm.mask_va, data.shape[:2], not_seen) != True] = np.nan
plt.matshow(predicted_labels_va, cmap=plt.cm.Accent, vmin=0, vmax=alpha.shape[1] - 1)
plt.title("validation nodes")
plt.show()

predicted_labels_te = imageify(alpha.argmax(axis=1), data.shape[:2], not_seen)
predicted_labels_te[imageify(sm.mask_te, data.shape[:2], not_seen) != True] = np.nan
plt.matshow(predicted_labels_te, cmap=plt.cm.Accent, vmin=0, vmax=alpha.shape[1] - 1)
plt.title("predictions for test nodes")
plt.show()

plt.matshow(imageify(sm[0].y.argmax(axis=1), data.shape[:2], not_seen), cmap=plt.cm.Accent, vmin=0, vmax=alpha.shape[1] - 1)
plt.title("GT labels")
plt.show()

right_vs_wrong = np.ones(alpha.shape[0:1] + (3,))
right_vs_wrong[alpha.argmax(axis=1) == sm[0].y.argmax(axis=1)] = [0, 1, 0]
right_vs_wrong[alpha.argmax(axis=1) != sm[0].y.argmax(axis=1)] = [1, 0, 0]
plt.imshow(imageify(right_vs_wrong, data.shape[:2], not_seen))
plt.title("Correct/Incorrect classification")
plt.show()

Visualize vacuity uncertainty (shown next to correct/incorrect classification to consider correlations)

In [ ]:
fig, ax = plt.subplots(1, 2)
cm = ax[0].matshow(imageify(get_vacuity(alpha), data.shape[:2], not_seen))
# fig.colorbar(cm, ax=ax[0])
ax[0].set_title("Vacuity uncertainty")
ax[1].imshow(imageify(right_vs_wrong, data.shape[:2], not_seen))
ax[1].set_title("Correct/Incorrect classification")
plt.show()

Visualize dissonance uncertainty (shown next to correct/incorrect classification to consider correlations)

In [ ]:
fig, ax = plt.subplots(1, 2)
cm = ax[0].matshow(imageify(dissonance_uncertainty(alpha), data.shape[:2], not_seen))
# fig.colorbar(cm, ax=ax[0])
ax[0].set_title("Dissonance uncertainty")
ax[1].imshow(imageify(right_vs_wrong, data.shape[:2], not_seen))
ax[1].set_title("Correct/Incorrect classification")
plt.show()